* [github salgo60/Svenskabadplatser](https://github.com/salgo60/Svenskabadplatser)
* this [Notebook](https://github.com/salgo60/Svenskabadplatser/blob/main/Jupyter/BathWater-Check.ipynb)

* file [bathing_waters_points_max100m_from_lake.geojson](https://raw.githubusercontent.com/salgo60/Svenskabadplatser/main/data/bathing_waters_points_max100m_from_lake.geojson) 
* [geopandas.org](https://geopandas.org/)

In [3]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-05-17 00:10:45.088856


In [8]:
import pandas as pd
import geopandas as gpd
file = "bathing_waters_points_max100m_from_lake.geojson"
dfgeo = gpd.read_file(file)
dfgeo.shape

(1675, 6)

In [10]:
type(dfgeo)

geopandas.geodataframe.GeoDataFrame

In [39]:
#dfgeo.info

In [40]:
dfgeo.head()

,nutsCode,SJOID,NAMN,VYNAMN,LW_PopNamn,geometry
0,SE0441273000000001,625683-137417,Vesljungasjön,Vesljungasjön,"88000_Vesljungasjön_(13.76,56.42)",POINT (423966.563 6253646.592)
1,SE0441273000000002,624815-138826,"Osbysjön, Ebbarps camping",Osbysjön,"88000_Osbysjön_(13.98,56.35)",POINT (438138.401 6246975.817)
2,SE0441273000000003,625269-140569,Hjärtasjöbadet,Hjärtasjön,"87000_Hjärtasjön_(14.28,56.4)",POINT (455735.712 6250820.500)
3,SE0441285000000004,620062-135224,"Ringsjön Sjöholmen, Västra str",Västra Ringsjön,"96000_Västra Ringsjön_(13.47,55.89)",POINT (402392.276 6196761.215)
4,SE0441265000000040,617666-135851,Östra Vombsjön,Vombsjön,"92000_Vombsjön_(13.59,55.68)",POINT (413540.395 6171563.249)


In [35]:
# Get all Swedish Bathing Waters with NUTS and if they are cinnected to a Water and if tyhat water has a SJOID
#
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
 
# https://w.wiki/3LWk    
queryBath = """SELECT (REPLACE(STR(?nodeBath), ".*Q", "Q") AS ?qbathid) ?nodeBathLabel ?nutsCode ?SJOID
(REPLACE(STR(?nodeWater), ".*Q", "Q") AS ?qlakeid) ?nodeWaterLabel WHERE {
  ?nodeBath wdt:P6104 wd:Q106774536.
  OPTIONAL { ?nodeBath wdt:P605 ?nutsCode. }
  OPTIONAL {
    ?nodeBath wdt:P206 ?nodeWater.
    OPTIONAL { ?nodeWater wdt:P761 ?SJOID. }
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY (?nodeBathLabel)"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDBath = get_sparql_dataframe(endpoint_url, queryBath)
WDBath["Source"] = "WD"     
WDBath.shape

(2752, 7)

In [36]:
WDBath

,qbathid,nodeBathLabel,nutsCode,SJOID,qlakeid,nodeWaterLabel,Source
0,Q106708773,Abborrbergets badplats,SE0220486000001903,None,Q35694946,Strängnäsfjärden,WD
1,Q106707080,Abborrsjöns badplats,SE0411060000000277,None,None,None,WD
2,Q106708954,Abborrtjärns badplats,SE0611763000002279,None,None,None,WD
3,Q106711299,Abborrtjärns badplats,SE0812404000003683,None,None,None,WD
4,Q106710481,Adaks badplats,SE0812418000003416,None,None,None,WD
...,...,...,...,...,...,...,...
2747,Q18335372,Österskärs havsbad,SE0110117000002087,None,None,None,WD
2748,Q106681296,Östnora havsbad,SE0110136000002146,None,None,None,WD
2749,Q106708690,"Östra Valsjöns badplats, Hedared",SE0A21490000001781,None,None,None,WD
2750,Q106710694,Östtjärn badplats,SE0712281000003466,691354-157517,Q18184573,Östtjärnen,WD


In [43]:
dfGeomerged = gpd.GeoDataFrame(pd.merge(WDBath, dfgeo, on="nutsCode", how="left"))

In [44]:
dfGeomerged

,qbathid,nodeBathLabel,nutsCode,SJOID_x,qlakeid,nodeWaterLabel,Source,SJOID_y,NAMN,VYNAMN,LW_PopNamn,geometry
0,Q106708773,Abborrbergets badplats,SE0220486000001903,None,Q35694946,Strängnäsfjärden,WD,658080-162871,"Strängnäsfjärden, Abborrberget",Mälaren-Gisselfjärden,"61000_Mälaren-Gisselfjärden_(16.96,59.43)",POINT (615733.433 6584650.820)
1,Q106707080,Abborrsjöns badplats,SE0411060000000277,None,None,None,WD,NaN,NaN,NaN,NaN,None
2,Q106708954,Abborrtjärns badplats,SE0611763000002279,None,None,None,WD,660121-136542,Abborrtjärn,Abborrtjärn,"108000_Abborrtjärn_(13.43,59.51)",POINT (411213.681 6597836.404)
3,Q106711299,Abborrtjärns badplats,SE0812404000003683,None,None,None,WD,713089-168959,Abborrtjärn,Abborrtjärnen,"28000_Abborrtjärnen_(19.73,64.23)",POINT (729029.740 7131312.139)
4,Q106710481,Adaks badplats,SE0812418000003416,None,None,None,WD,725253-162920,Adak,Stor-Adakträsket,"20000_Stor-Adakträsket_(18.57,65.35)",POINT (666363.075 7252654.435)
...,...,...,...,...,...,...,...,...,...,...,...,...
2747,Q18335372,Österskärs havsbad,SE0110117000002087,None,None,None,WD,NaN,NaN,NaN,NaN,None
2748,Q106681296,Östnora havsbad,SE0110136000002146,None,None,None,WD,NaN,NaN,NaN,NaN,None
2749,Q106708690,"Östra Valsjöns badplats, Hedared",SE0A21490000001781,None,None,None,WD,641433-131980,"Östra Valsjöns badplats, Hedared",Östra Valsjön,"106000_Östra Valsjön_(12.78,57.82)",POINT (368338.246 6410511.817)
2750,Q106710694,Östtjärn badplats,SE0712281000003466,691354-157517,Q18184573,Östtjärnen,WD,691354-157517,Östtjärn Matfors,Öst-Tjärnen,"42000_Öst-Tjärnen_(17.26,62.33)",POINT (616770.133 6912658.437)


In [47]:
type(dfGeomerged)

geopandas.geodataframe.GeoDataFrame

In [48]:
dfmerged = pd.DataFrame(dfGeomerged)
type(dfmerged)

pandas.core.frame.DataFrame

In [51]:
dfmerged

,qbathid,nodeBathLabel,nutsCode,SJOID_x,qlakeid,nodeWaterLabel,Source,SJOID_y,NAMN,VYNAMN,LW_PopNamn,geometry
0,Q106708773,Abborrbergets badplats,SE0220486000001903,None,Q35694946,Strängnäsfjärden,WD,658080-162871,"Strängnäsfjärden, Abborrberget",Mälaren-Gisselfjärden,"61000_Mälaren-Gisselfjärden_(16.96,59.43)",POINT (615733.433 6584650.820)
1,Q106707080,Abborrsjöns badplats,SE0411060000000277,None,None,None,WD,NaN,NaN,NaN,NaN,None
2,Q106708954,Abborrtjärns badplats,SE0611763000002279,None,None,None,WD,660121-136542,Abborrtjärn,Abborrtjärn,"108000_Abborrtjärn_(13.43,59.51)",POINT (411213.681 6597836.404)
3,Q106711299,Abborrtjärns badplats,SE0812404000003683,None,None,None,WD,713089-168959,Abborrtjärn,Abborrtjärnen,"28000_Abborrtjärnen_(19.73,64.23)",POINT (729029.740 7131312.139)
4,Q106710481,Adaks badplats,SE0812418000003416,None,None,None,WD,725253-162920,Adak,Stor-Adakträsket,"20000_Stor-Adakträsket_(18.57,65.35)",POINT (666363.075 7252654.435)
...,...,...,...,...,...,...,...,...,...,...,...,...
2747,Q18335372,Österskärs havsbad,SE0110117000002087,None,None,None,WD,NaN,NaN,NaN,NaN,None
2748,Q106681296,Östnora havsbad,SE0110136000002146,None,None,None,WD,NaN,NaN,NaN,NaN,None
2749,Q106708690,"Östra Valsjöns badplats, Hedared",SE0A21490000001781,None,None,None,WD,641433-131980,"Östra Valsjöns badplats, Hedared",Östra Valsjön,"106000_Östra Valsjön_(12.78,57.82)",POINT (368338.246 6410511.817)
2750,Q106710694,Östtjärn badplats,SE0712281000003466,691354-157517,Q18184573,Östtjärnen,WD,691354-157517,Östtjärn Matfors,Öst-Tjärnen,"42000_Öst-Tjärnen_(17.26,62.33)",POINT (616770.133 6912658.437)


In [ ]:
def f(x):    
   return 'same' if x['SJOID_x'] = x['SJOID_y'] else 'not same'

df['is_score_chased'] = df.apply(f, axis=1)
# Find Ok
dfOk = dfmerged 

In [ ]:
# Find mismatch

In [ ]:
# Find missing

#Join